# 12.3. Deflategate

- 2015 American Football Conference (AFC) championship
- allegations: the Patriots’ footballs had not been inflated as much as the regulations required; they were softer and might be easier to catch.
- game balls must be inflated to have pressures in the range 12.5 psi and 13.5 psi
- Before the start of the AFC game, all the Patriots’ balls were at about 12.5 psi. Most of the Colts’ balls were at about 13.0 psi. (not recorded)
- During the second quarter, the Colts intercepted a Patriots ball. On the sidelines, they measured the pressure of the ball and determined that it was below the 12.5 psi threshold. Promptly, they informed officials.
- At half-time, all the game balls were collected for inspection. Two officials, Clete Blakeman and Dyrol Prioleau, measured the pressure in each of the balls.



In [ ]:
from datascience import *
path_data = '../../data/'  
football = Table.read_table(path_data + 'deflategate.csv')

### footballs: 11 from the Patriats and 4 from the Colts
### measured by two officials
football.show()

In [ ]:
### combined PSI
football = football.with_column(
  'Combined', ( football.column(1) + football.column(2) ) / 2)
football.show()

In [ ]:
### combined only
football = football.drop(1, 2)
football.show()

### Patriots balls are at lower pressure?

In [ ]:
### some deflation is normal during the course of a game
### calculating the pressure drop:
##### setting up the starting values
##### use the NumPy function np.ones()
import numpy as np

np.ones(11)

In [ ]:
##### create the ideal PSI
patriots_start = 12.5 * np.ones(11)
colts_start = 13 * np.ones(4)
start = np.append(patriots_start, colts_start)
start

In [ ]:
### pressure drop from pre-game measures
drop = start - football.column('Combined')
drop

In [ ]:
football = football.with_column('Pressure Drop', drop)
football
football.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
football.hist('Pressure Drop', group='Team')

In [ ]:
### It looks as though the Patriots’ drops were larger than the Colts’. 
### Let’s look at the average drop in each of the two groups. 
### We no longer need the combined scores.

football = football.drop('Combined')
football.group('Team', np.average)   ### group by team, return function

- why the **Patriots’ footballs** had a **larger drop** in pressure, on average, than the Colts footballs. 
- Could it be due to **chance**?

## 12.3.1. The Hypotheses

- **Null hypothesis**: we can make a chance model by **hypothesizing** that the 11 Patriots’ drops look like a **random sample of 11 out of all the 15 drops**, with the Colts’ drops being the remaining four.
- **Alternative hypothesis**: the Patriots’ drops are **too large, on average, to resemble a random sample** drawn from all the drops.

## 12.3.2. Test Statistic

- A natural statistic is **the difference between the two average drops** to be calculated as the **"average drop for Patriots - average drop for Colts"**.

In [ ]:
### group gives count, we throw in an average function

observed_means = football.group('Team', np.average).column(1)   
observed_means   ##### means of the teams' drops

In [ ]:
##### use the item
##### item means the element at the specified index

observed_difference = observed_means.item(1) - observed_means.item(0)
observed_difference

#### mean difference

In [ ]:
### put our thoughts together to create a function
### function to calculate the difference between the mean drops of the two groups

def difference_of_means(table, group_label):               ### label: T/F, Patriats/Colts
    reduced = table.select('Pressure Drop', group_label)   ### reduced table: team & drops
    means_table = reduced.group(group_label, np.average)   ### average out the drops for the teams
    means = means_table.column(1)                          ### choose the column (arry of averages)
    return means.item(1) - means.item(0)                   ### calc the distance

In [ ]:
difference_of_means(football, 'Team')

## 12.3.3. Predicting the Statistic Under the Null Hypothesis

- If the null hypothesis were true, then it shouldn’t matter which footballs are labeled Patriots and which are labeled Colts. ==> The distributions of the two sets of drops would be the same. We can simulate this by randomly shuffling the team labels.

In [ ]:
##### permutation: shuffle the team labels
##### one simulation

shuffled_labels = football.sample(with_replacement=False).column(0)

In [ ]:
original_and_shuffled = football.with_column('Shuffled Label', shuffled_labels)
original_and_shuffled   ### the table to be used

In [ ]:
##### compare group means

difference_of_means(original_and_shuffled, 'Shuffled Label')

In [ ]:
### compare to the observed difference

difference_of_means(original_and_shuffled, 'Team')

**The two teams’ average drop values are closer when the team labels are randomly assigned** to the footballs than they were for the two groups actually used in the game.

## 12.3.4. Permutation Test

- repeatedly simulate of the test statistic under the null hypothesis.
- start by writing a function one_simulated_difference that returns one simulated value of the difference between the mean pressure drops of the groups labeled Patriots and Colts after randomly permuting the team labels of the footballs.

In [ ]:
def one_simulated_difference():
    shuffled_labels = football.sample(with_replacement=False).column('Team')   ### shuffle the labels (Team)
    shuffled_table = football.select('Pressure Drop').with_column(
    'Shuffled Label', shuffled_labels)                                          ### two cols ==> table
    return difference_of_means(shuffled_table, 'Shuffled Label')

In [ ]:
differences = make_array()
repetitions = 1000

for i in np.arange(repetitions):
    new_difference = one_simulated_difference()
    differences = np.append(differences, new_difference)

In [ ]:
differences

## 12.3.5. Conclusion of the Test


In [ ]:
import matplotlib.pyplot as plots
%matplotlib inline

Table().with_column(
    'Difference Between Group Means', differences).hist(bins = 50)

plots.ylim(-0.1, 1.4)
plots.scatter(observed_difference, 0, color='red', s=30, zorder=3)
plots.title('Prediction Under the Null Hypothesis')
print('Observed Difference:', observed_difference)

In [ ]:
empirical_p = np.count_nonzero(differences >= observed_difference) / 10000
empirical_p

- As in previous examples of this test, the bulk of the distribution is centered around 0. Under the null hypothesis, the Patriots’ drops are a random sample of all 15 drops, and therefore so are the Colts’. Therefore the two sets of drops should be about equal on average, and therefore their difference should be around 0.
- But the observed value of the test statistic is quite far away from the heart of the distribution. By any reasonable cutoff for what is “small”, the empirical P-value is small. So we end up rejecting the null hypothesis of randomness, and conclude that the Patriots drops were too large to reflect chance variation alone.

- The independent investigative team analyzed the data in several different ways, taking into account the laws of physics. The final report said,

- “[T]he average pressure drop of the Patriots game balls exceeded the average pressure drop of the Colts balls by 0.45 to 1.02 psi, depending on various possible assumptions regarding the gauges used, and assuming an initial pressure of 12.5 psi for the Patriots balls and 13.0 for the Colts balls.” – Investigative report commissioned by the NFL regarding the AFC Championship game on January 18, 2